In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

In [11]:
import tensorflow as tf

#não estou conseguindo fazer funcionar, mas nem é necessário já q são dados tabulares e são poucos.
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        #não sei pra q serve mas tá escrito q isso faz com que o TensorFlow pegue memória aos poucos, e não toda de uma vez.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"✅ GPU Encontrada: {gpus}")
        print("O treinamento será acelerado pela placa de vídeo!")
        
    except RuntimeError as e:
        print(e)
else:
    print("❌ Nenhuma GPU detectada. O treinamento será feito na CPU (mais lento).")

❌ Nenhuma GPU detectada. O treinamento será feito na CPU (mais lento).


In [12]:
#carregando o banco de dados e tirando os espaços pq é foda né
df = pd.read_csv('paddydataset.csv')
df.columns = df.columns.str.strip()

In [13]:
#separar Features (X) selecionadas e alvo (y)
#por enquanto o alvo é a última coluna: 'Paddy yield(in Kg), talvez mudar?'
features = [
    'Hectares', 'Variety', 'Soil Types', 
    'Seedrate(in Kg)', 'Nursery area (Cents)', 'LP_Mainfield(in Tonnes)',
    'DAP_20days', 'Urea_40Days', 'Potassh_50Days', 'Micronutrients_70Days',
    'Weed28D_thiobencarb', 'Pest_60Day(in ml)'
]
target = 'Paddy yield(in Kg)'

X = df[features]
y = df[target]

In [14]:
#pré-processamento Identificar colunas numéricas e categóricas, tava escrito pra fazer só fiz
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [15]:
#processamento
#identificar tipos automaticamente
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['number']).columns

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_processed = preprocessor.fit_transform(X)

In [16]:
#clásico sem mt firula dividindo em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [17]:
#converter para denso se necessário não sei exatamente oq faz mas to com medo de tirar
if hasattr(X_train, "toarray"): X_train = X_train.toarray()
if hasattr(X_test, "toarray"): X_test = X_test.toarray()

In [18]:
#multilayer perceptron bem basiquinha sem mt presepada
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

#avaliação
loss, mae = model.evaluate(X_test, y_test)
print(f"Novo Erro Médio (MAE): {mae:.2f} Kg")

#salvar
model.save('modelo_paddy_compacto.h5')

Epoch 1/50


C:\Users\renat\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 590074560.0000 - mae: 22463.2207  
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - loss: 584722368.0000 - mae: 22346.4199
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss: 549610432.0000 - mae: 21556.1895
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - loss: 439636928.0000 - mae: 18995.9668
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - loss: 257638960.0000 - mae: 14629.4111
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 105740096.0000 - mae: 9140.8096
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - loss: 56192360.0000 - mae: 6340.8608
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 45239696.0000 - mae: 5351.8662
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: 37321948.0000 - mae: 4726.4556
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32074306.0000 - mae: 4367.3027
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - loss: 28731102.0000 - mae: 4144.7188
E

Novo Erro Médio (MAE): 678.76 Kg
